# Basic CSP Startup Script

Before starting the notebook, please run the job `deploy-sut-integration` of the `integration` stage in the latest gitlab pipeline: https://gitlab.com/ska-telescope/ska-mid-itf/-/pipelines?page=1&scope=all&ref=main. 

<div class="alert alert-block alert-danger">
<b>WARNING</b>The CBF configuration scripts' output is extremely verbose and will cause the CI job outputs to be too large to download in a single CI job. In future this might be improved.</div>

In [ ]:
import os
from tango import DeviceProxy, Database

# Take the namespace name from the deployment job
KUBE_NAMESPACE = "integration"
CLUSTER_DOMAIN = "miditf.internal.skao.int"
# set the name of the databaseds service
DATABASEDS_NAME = "tango-databaseds"

# finally set the TANGO_HOST
os.environ[
    "TANGO_HOST"
] = f"{DATABASEDS_NAME}.{KUBE_NAMESPACE}.svc.{CLUSTER_DOMAIN}:10000"

List all devices available:

In [ ]:
database = Database()
instance_list = database.get_device_exported("*")
for instance in instance_list.value_string:
    dev = DeviceProxy(instance)
    print(instance)
    print("*****")
    print(dev.info())

## Setup Device connection and Timeouts
Set up DeviceProxy, check AdminMode and if not 0, set to 0 (ONLINE)

In [ ]:
CSP = DeviceProxy("mid-csp/control/0")

In [ ]:
CSP.adminmode # This is likely currently OFFLINE

In [ ]:
CSP.State() # This should now be DISABLE

Set Adminmode to ONLINE and check state

In [ ]:
CSP.adminmode=0
CSP.adminmode

In [ ]:
CSP.State() # This should now be OFF

Check CBF SimulationMode (this should be FALSE for real hardware control)

In [ ]:
CSP.cbfSimulationMode # Should be True if hardware is not currently under CSP.LMC control

In [ ]:
CSP.cbfSimulationMode=False
CSP.cbfSimulationMode

Set Timeout to 45 seconds as the ON command is a long-running command and check value

In [ ]:
CSP.commandTimeout=45
CSP.commandTimeout

## Send ON command
Send the ON command. In the current configuration, this requires the CBF to do some things.

NOTE: an empty list sends the ON command to ALL the subsystems, specific subsystems are turned on if specified in a list of subsystem FQDNs

In [ ]:
CSP.on([])

In [ ]:
CSP.State() # this should now be ON

## Check all relevant device states
Now check if all the devices are switched on as they should be.
Use the [Taranta Dashboard](https://k8s.miditf.internal.skao.int/integration/taranta/dashboard?id=6543b36cf6d53e0019b95cf1&mode=run) to monitor top-level devices!

## Teardown
Turn off the CSP (as it is now the end of the test session!), set its AdminMode to OFFLINE, reset the Timeout, set CBFSimulationMode back to True so other people can play with this device.

In [ ]:
CSP.off([]) # Check with make itf-cbf-talonlru-status - lru should be off now
CSP.cbfSimulationMode=True
CSP.commandTimeout=3
CSP.adminmode=1